In [11]:
from dotenv import load_dotenv
import os


Config the LLM

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')
output = model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


 Config the embedding model


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

c:\CSE-AI\GEN-AI\genai&agenticai\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384

Config the Document loader

In [14]:
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain_community.vectorstores import chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
loader = DirectoryLoader("../data",glob="./*.txt",loader_cls=TextLoader)
docs = loader.load()
docs

[Document(metadata={'source': '..\\data\\sample.txt'}, page_content="🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which 

In [16]:
docs[0].page_content

"🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which ranks 2nd). The U.S. GDP per capita is also among the highest, hover

config the Text spliiting / chunking 

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap=50
)
new_docs = text_splitter.split_documents(documents=docs)

In [18]:
new_docs

[Document(metadata={'source': '..\\data\\sample.txt'}, page_content='🇺🇸 Overview of the U.S. Economy'),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content='The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,'),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content='It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a'),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content='a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.'),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content='innovation, glo

In [19]:
doc_string = [doc.page_content for doc in new_docs]
doc_string

['🇺🇸 Overview of the U.S. Economy',
 'The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,',
 'It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a',
 'a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.',
 'innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.',
 'U.S. GDP – Size, Composition, and Global Share',
 'As of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 

Config vector DB 

In [21]:
from langchain_chroma import Chroma
db=Chroma.from_documents(new_docs,embeddings)
retriever = db.as_retriever(search_kwargs={"k":3})
retriever.invoke("idustrial growth of usa?")

[Document(id='6f66c5ab-a61a-4dbc-975c-f71f78f3800f', metadata={'source': '..\\data\\sample.txt'}, page_content='🇺🇸 Overview of the U.S. Economy'),
 Document(id='5febc263-9a3c-4d0d-a77c-6e48e642d4b3', metadata={'source': '..\\data\\sample.txt'}, page_content='Looking forward, the U.S. economy is expected to grow at a moderate pace, powered by innovation in AI, green energy, robotics, biotech, and quantum computing. The Biden administration’s Inflation'),
 Document(id='66e4e367-716a-4584-ac80-4c869488ca37', metadata={'source': '..\\data\\sample.txt'}, page_content='GDP Growth Trends and Dynamics')]

Creation of pydatic class


In [22]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END
from langchain.output_parsers import PydanticOutputParser

In [23]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description="Reasoning behind topic selection")

In [39]:
parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

This below AgentState is just for the explation like how state works

In [26]:
Agentstate={}

In [27]:
Agentstate['messages'] = []

In [28]:
Agentstate

{'messages': []}

In [29]:
Agentstate["messages"].append("what are you doing ")

In [30]:
Agentstate

{'messages': ['what are you doing ']}

In [31]:
Agentstate["messages"].append("I hope everything fine")

In [32]:
Agentstate

{'messages': ['what are you doing ', 'I hope everything fine']}

In [33]:
Agentstate["messages"][-1]

'I hope everything fine'

In [34]:
Agentstate["messages"][0]

'what are you doing '

## AgentState workflow

This AgentState class you need to inside the stategraph

In [35]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],operator.add]



In [36]:
state = {"message":["hi"]}

In [37]:
state = "hi"

In [41]:
def function_1(state:AgentState):

    question = state["messages"][-1]

    print("Question",question)

    template=""" 
    Your task is to classify the given user query into one of the following categories: [USA,Not realted].
    only respond with the category name and nothing else.


    user query : {question}
    {format_instruction}
    """


    prompt = PromptTemplate(
        template=template,
        input_variables=["question"],
        partial_variables={"format_instruction": parser.get_format_instructions}
    )

    chain = prompt | model | parser

    response = chain.invoke({"question":question})

    print("parsed responce: ",response)

    return {"messages":[response.Topic]}

